In [1]:
import os
import sys
import shutil
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import pickle

from tqdm import tqdm

# Root directory of the project
ROOT_DIR = os.path.abspath("./Mask_RCNN")

import warnings
warnings.filterwarnings("ignore")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline

Using TensorFlow backend.


## Attention ces cellules ont été exécutées en utilisant tensorflow-gpu et la fonctionnalité Nvidia CUDA pour avoir un temps d'exécution passant de 180h (avec tensorflow-cpu) à 8h

In [2]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join('', "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join("./images")

In [3]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.525

config = InferenceConfig()

In [4]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir='mask_rcnn_coco.hy', config=config)

# Load weights trained on MS-COCO
model.load_weights('mask_rcnn_coco.h5', by_name=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [5]:
# COCO Class names
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [6]:
files = os.listdir('./images')

In [7]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14497312495509646053
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4965466112
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7804349373482791636
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [8]:
for i in tqdm(range(len(files))):
    try:
        # Load a random image from the images folder
        file = files[i]
        pfile = './images/'+file
        image = skimage.io.imread(pfile)
        # Run detection
        results = model.detect([image], verbose=0)
        r = results[0]
        mask = r['masks']

        fmask = np.zeros((mask.shape[0],mask.shape[1]))

        for i in range(mask.shape[0]):
            for j in range(mask.shape[1]):
                for k in range(mask.shape[2]):
                    fmask[i][j] += mask[i][j][k]

        fmask = fmask.astype(bool)
        fmask = fmask.astype(np.uint8)

        temp = skimage.io.imread(pfile)
        for i in range(temp.shape[2]):
            temp[:,:,i] *= fmask[:,:]
        skimage.io.imsave('./masked_images/'+file,temp)
    except:
        try:
            file = files[i]
            dest = './masked_images/'+file
            shutil.copy(file,dest)
        except : pass

  0%|                                                                                       | 0/249470 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████| 249470/249470 [9:15:45<00:00,  7.48it/s]
